In [ ]:
import gradio as gr
from dslmodel import DSLModel, init_instant
from pydantic import Field
from typing import Dict
import openai

# Initialize DSL instant
init_instant()

# %%
class CoverLetterModel(DSLModel):
    """
    Represents a professional cover letter.
    """
    sender_name: str = Field(..., description="Full name of the sender.")
    sender_contact: Dict[str, str] = Field(..., description="Contact information of the sender.")
    recipient_name: str = Field(..., description="Full name of the recipient.")
    recipient_position: str = Field(..., description="Position title of the recipient.")
    company_name: str = Field(..., description="Name of the recipient's company.")
    company_address: str = Field(..., description="Address of the recipient's company.")
    subject: str = Field(..., description="Subject line of the cover letter.")
    body: str = Field(..., description="Main content of the cover letter.")
    closing: str = Field(..., description="Closing remarks of the cover letter.")
    signature: str = Field(..., description="Sender's signature.")

# %%
# Function to generate cover letter using OpenAI's GPT-3.5
def generate_cover_letter(input_text):
    """
    Generates a professional cover letter based on the provided job description.
    """
    if not input_text.strip():
        return "Please paste a job description to generate a cover letter."

    openai.api_key = 'your-openai-api-key'  # Replace with your actual API key

    prompt = f"Generate a professional cover letter based on the following job description:\n\n{input_text}"

    CoverLetterModel.from_prompt(prompt)

# %%
# Define the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Job Application Assistant")
    gr.Markdown(
        """
        This interface allows you to:
        1. **Paste** a job description into the input box.
        2. **Generate** a cover letter based on the job description.
        3. **Download** the generated cover letter as a text file.
        """
    )
    
    with gr.Row():
        input_textbox = gr.Textbox(
            label="Job Description",
            lines=10,
            placeholder="Paste the job description here...",
            interactive=True
        )
    
    with gr.Row():
        generate_button = gr.Button("Generate Cover Letter")
    
    with gr.Row():
        output_textbox = gr.Textbox(
            label="Generated Cover Letter",
            lines=10,
            interactive=False
        )
    
    with gr.Row():
        download_button = gr.File(label="Download Cover Letter")
    
    # %%
    # Define the function to handle download
    def prepare_download(cover_letter):
        """
        Prepares the cover letter text as a downloadable file.
        """
        if "Error" in cover_letter or "Please paste" in cover_letter:
            return None  # Do not provide a download if there's an error or no input
        return gr.File.update(value=None, filename="Cover_Letter.txt", file=cover_letter.encode('utf-8'))
    
    # %%
    # Define the interactions
    generate_button.click(
        fn=generate_cover_letter,
        inputs=input_textbox,
        outputs=output_textbox
    )
    
    # Update the download button after generating the cover letter
    generate_button.click(
        fn=lambda cover_letter: cover_letter,
        inputs=output_textbox,
        outputs=download_button
    )
    
    # %%
    # Optional: Clear download button if cover letter is not generated properly
    input_textbox.change(
        fn=lambda _: None,
        inputs=None,
        outputs=download_button
    )

# %%
# Launch the Gradio app
demo.launch()
